<a id='top'></a>
# Anneal Offsets
### Advanced `optimize_binary` usage


## Table of Contents
1. [Introduction](#introduction)
2. [Problem Definition](#problem_definition)
3. [Using Anneal Offsets](#anneal_offsets)
4. [Further Documentation and Support](#doc_and_support)


## 1. Introduction <a id="introduction"></a>
In this notebook, we will review an advanced option of QCWare's `optimize_binary` function. The D'Wave QPU allows for [Anneal Offsets](https://docs.dwavesys.com/docs/latest/c_fd_ao.html), which are slight variations in the anneal start time for each qubit. In this notebook, we will show how to use the anneal offsets functionality through the QC Ware Platform. We will discuss a [method](https://arxiv.org/abs/1806.11091) that we developed for generating anneal offsets that often improves the performance of the quantum annealer and show how to use this method through our platform.

First we'll import the QC Ware library and set our API key if necessary (you can find your API key on you [Forge](https://forge.qcware.com) dashboard.)

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('anneal_offsets.ipynb')


## 2. Problem Definition <a id="problem_definition"></a>

Throughout this notebook, we will use the Alternating Sectors Chain (ASC) problem as an example. It is a very simple problem with trivial solutions and contains features that cause the quantum annealer to exhibit interesting behavior, thus making it a good example to study.

The ASC problem requires four inputs;
1. $N$, number of spins ($N \in \mathbb{Z}^{+}$),
2. $\ell$, chain-length ($\ell \in \mathbb{Z}, \ 1 \leq \ell \leq N$),
3. $w$, weak coupling ($w \in \mathbb{R}, \ 0 \leq w \leq S$), and
4. $S$, strong coupling ($S \in \mathbb{R}, \ w \leq S$).

We define these below.

In [2]:
N = 40
l = 4
w = 1
S = 2

# check problem definition is valid
assert isinstance(N, int)
assert isinstance(l, int)
assert N >= 1
assert 1 <= l <= N
assert 0 <= w <= S

The ASC problem then becomes finding 

$$z^* = \underset{z \in \{-1, 1 \}^N}{\operatorname{argmin}} H$$

where 

$$H = \sum_{i=0}^{N-2} J_{i, i+1}z_i z_{i+1}, \qquad J_{i, i+1} = \begin{cases}
-w&\text{if }\left\lfloor \frac{i}{\ell} \right\rfloor \text{ is odd} \\
-S&\text{else}
\end{cases}$$

Notice that the solution is two-fold degenerate, with simply all of the spins aligned;

$$z_i = -1 \ \forall i \in \{0, 1, ..., N-1 \} \qquad {\rm or} \qquad z_i = 1 \ \forall i \in \{0, 1, ..., N-1 \}$$

This is an instance of a more general problem called the [Ising model](https://en.wikipedia.org/wiki/Ising_model). There is a one-to-one mapping between Quadratic Unconstrained Binary Optimization (QUBO) and Ising problems. 

Let $x = \frac{z+1}{2}$, or $z = 2x-1$; then $x\in \{0, 1 \}^N$. We can reformulate $H$ in terms of $x$ which will transform it into a QUBO formulation.
\begin{align*}
H &= \sum_{i=0}^{N-2} J_{i, i+1}z_i z_{i+1}\\
& = \sum_{i=0}^{N-2} J_{i, i+1}(2x_i-1)(2x_{i+1}-1)\\
&= \sum_{i=0}^{N-1}\sum_{i'=0}^{N-1} Q_{ii'}x_ix_{i'} + {\rm offset}
\end{align*}
where
$${\rm offset} = \sum_{i=0}^{N-2}J_{i,i+1}  \qquad  Q_{ii'} = \begin{cases}
4J_{ii'}&\text{if } i' = i+1\\
-2J_{0,1}&\text{if } i = i' = 0\\
-2J_{N-2,N-1}&\text{if } i = i' = N-1\\
-2(J_{i-1,i}+J_{i, i+1})&\text{if } i = i'\\
0&{\rm else}
\end{cases}$$

Thus, $Q$ is our QUBO formulation, and the two solutions are now

$$x_i = 0 \ \forall i \in \{0,1,...,N-1 \} \qquad {\rm or} \qquad x_i = 1 \ \forall i \in \{0,1,...,N-1 \}$$

Let's create the QUBO dictionary.

In [3]:
def J(i, ip):
    """ define the Ising couplings """
    if ip != i + 1 or not 0 <= i < N or not 0 <= ip < N:
        return 0
    elif (i // l) % 2:  # odd
        return -w
    else:  # even
        return -S

def q(i, ip):
    """ define the QUBO values """
    if not 0 <= i < N or not 0 <= ip < N:
        return 0
    elif ip == i + 1:
        return 4 * J(i, ip)
    elif i == ip == 0:
        return -2 * J(0, 1)
    elif i == ip == N - 1:
        return -2 * J(N - 2, N - 1)
    elif i == ip:
        return -2 * (J(i - 1, i) + J(i, i + 1))
    else:
        return 0

# create the QUBO dictionary
Q, offset = {}, 0
for i in range(N):
    offset += J(i, i + 1)
    v = q(i, i)
    if v: Q[(i, i)] = v
    v = q(i, i + 1)
    if v: Q[(i, i + 1)] = v
        
# show two 'chains' of the QUBO (and the start of the third chain).
print("First two chains (and start of the third) of the QUBO")
for i in range(2*l+1):
    print("Q(%d, %d) = %g" % (i, i, Q.get((i, i), 0)))
    print("Q(%d, %d) = %g" % (i, i+1, Q.get((i, i+1), 0)))

First two chains (and start of the third) of the QUBO
Q(0, 0) = 4
Q(0, 1) = -8
Q(1, 1) = 8
Q(1, 2) = -8
Q(2, 2) = 8
Q(2, 3) = -8
Q(3, 3) = 8
Q(3, 4) = -8
Q(4, 4) = 6
Q(4, 5) = -4
Q(5, 5) = 4
Q(5, 6) = -4
Q(6, 6) = 4
Q(6, 7) = -4
Q(7, 7) = 4
Q(7, 8) = -4
Q(8, 8) = 6
Q(8, 9) = -8


For covenience, let's define a function that checks whether or not we found one of the two correct solutions.

In [4]:
def is_correct_solution(solution):
    return all(solution[i] == 0 for i in range(N)) or all(solution[i] == 1 for i in range(N))

## 3. Using Anneal Offsets <a id="anneal_offsets"></a>

QC Ware's `optimize_binary` functionality allows low-level usage of the anneal offsets functionality through the `dwave_anneal_offsets` keyword argument where a list of offsets can be supplied and applied to the QPU (for details on applying anneal offsets to D'Wave's quantum annealer, see their [documentation](https://docs.dwavesys.com/docs/latest/c_fd_ao.html)). However, choosing anneal offsets is very challenging (requires 2048 specific qubit offsets!) and is indeed an ongoing research task. Researchers at QC Ware have developed a powerful heuristic for determining good anneal offset values given a particular QUBO. The heuristic requires a single input labeled $\delta$, where $\delta \geq 0$ (in practice, $\delta$ should be in $[0, 0.05]$). Given $\delta$, we generate offsets for each qubit. We have shown our method to often give a performance advantage over standard quantum annealing; we will use the ASC chain example that we set up in [Section 2](#problem_definition) to show this.

We will look at the success probability metric to analyze performance. We define the success probability to be $\frac{c}{n}$, where $n$ is the number of times we have run our optimization algorithm and $c$ is the number of time we have found a correct solution (one of the two solutions mentioned above; all the variables equal to each other). Let's create a function to find the success probability for solving the ASC problem created above. We can supply a $\delta$ to `optimize_binary` with the `dwave_anneal_offsets_delta` keyword.

Since this process can take a while, with multiple calls to the annealer, let's use the Python library `tqdm` to display a progress bar

In [5]:
!pip install tqdm
from tqdm.notebook import tnrange, tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 13.0 MB/s eta 0:00:00


In [6]:
import qcware.types
def find_success_probability(n=20, delta=None):
    """ n is the number of runs, if delta is None, then we do not use anneal offsets """
    
    c = 0
    for _ in tnrange(n, desc=f"Delta={delta}"):
        # transform our Q into a BinaryProblem class
        objective = qcware.types.optimization.PolynomialObjective(
            polynomial=Q,
            num_variables=N,
            domain='boolean'
        )
        problem = qcware.types.optimization.BinaryProblem(objective=objective)
        # call qcware's optimize_binary function for solving QUBOs
        res = forge.optimization.optimize_binary(instance=problem, backend='dwave/2000q', dwave_anneal_offsets_delta=delta)
        # check if we found a correct solution
        if is_correct_solution(res.lowest_value_bitstring):
            c += 1
    
    return c / n

Finally, let's compare the performances! We will solve the ASC problem for various values of $\delta$ and see how the success probability changes. **The following cell will take few minutes to complete.** *Note: we have found that using our anneal offsets method tends to give significantly better results when solving the ASC problem than standard quantum annealing; however, quantum annealing is inherently probabilistic and therefore may not always give the same results. Run this notebook a few times, or increase the value of `n` in the `find_success_probability` function to get better statistics!*

Note: this cell is commented out by default because it can take over 20-30 minutes to run--just uncomment it to try it!

In [7]:
# standard = find_success_probability()
# print("Standard quantum annealing found a success probability of", standard)
# forge.config.set_client_timeout(20*60)
# for delta in (.01, .02, .03, .04, .05):
#     sp = find_success_probability(delta=delta)
#     print("The anneal offsets heuristic with delta =", delta, "found a success probability of", sp)

## 4. Further Documentation and Support  <a id="doc_and_support"></a>
Please see our [publication](https://arxiv.org/abs/1806.11091) for a detailed description of our anneal offsets method.

For more examples of how to use the platform to solve real-world problems, please take a moment to look through our demos. We recommend exploring these demos as a next step towards harnessing the power of the quantum cloud for your applications.

Complete documentation for all functions and parameters is available at https://qcware.readthedocs.io/en/latest/. For further support, please do not hesitate to contact the QC Ware team at support@qcware.com.

<a href="#top">Back to Table of Contents</a>